In [ ]:
import numpy as np
import pandas as pd
import PIL
import zipfile
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import random

MAIN_PATH = r"C:\Users\susmi\Downloads\img_align_celeba\img_align_celeba"  # Define the main path where images are stored

In [ ]:
image_paths = glob(MAIN_PATH+"/*") # Get all image file paths from the dataset folder

In [ ]:
len(image_paths) # Count total number of images

In [ ]:
# Load CelebA attributes dataset 
zip_path = r"C:\Users\susmi\Downloads\list_bbox_celeba.csv.zip"
attributes_df = pd.read_csv(zip_path, compression='zip')
attributes_df.head() # Show first few rows of attributes

In [ ]:
# Load landmarks dataset
zip_path = r"C:\Users\susmi\Downloads\list_landmarks_align_celeba.csv.zip"
landmarks_df = pd.read_csv(zip_path, compression='zip')
landmarks_df.head() # Show first few rows

In [ ]:
# Load partition dataset 
zip_path = r"C:\Users\susmi\Downloads\list_eval_partition.csv.zip"
partition = pd.read_csv(zip_path, compression='zip')
partition.head() # Show first few rows

In [ ]:
partition['partition'].value_counts() # Count how many images belong to each partition

In [ ]:
# Split dataset into train, validation, and test sets
train_images = partition.query('partition == 0')
valid_images = partition.query('partition == 1')
test_images = partition.query('partition == 2')

In [ ]:
#Code For Discriminator
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),    # Input image 64x64 RGB
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"), # Conv layer
        layers.LeakyReLU(negative_slope=0.2), # Activation function
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Flatten(),  # Flatten into 1D
        layers.Dropout(0.2),   # Prevent overfitting
        layers.Dense(1, activation="sigmoid"),   # Output: real/fake
    ],
    name="discriminator",
)
discriminator.summary()  # Show model summary

In [ ]:
#code for generator

latent_dim = 100  # Random noise dimension

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)), # Input: noise vector
        layers.Dense(8 * 8 * 512),  
        layers.Reshape((8, 8, 512)),   # Reshape to 8x8x512
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)

generator.summary()   # Show model summary

In [ ]:
# Load images from directory and resize to 64x64
dataset = keras.preprocessing.image_dataset_from_directory("archive", label_mode=None, image_size=(64, 64), batch_size=32
)
dataset = dataset.map(lambda x: x / 255.0) # Normalize pixel values to [0, 1]

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")
        
    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
 # Save generated images after each epoch
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255  # Rescale for saving
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))  # Save image
            plt.savefig('image_at_epoch_{:04d}.png'.format(epoch)) # Save plot
            plt.show()

In [ ]:
epochs = 10  # Number of epochs

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim) # Build GAN model

# Compile with Adam optimizer and Binary Cross-Entropy loss
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

# Train GAN on dataset
gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
# Display all generated images
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = []
for img_path in glob.glob('Generated Faces/*.png'):  # Load all generated images
    images.append(mpimg.imread(img_path))
    
# Plot generated images in grid
plt.figure(figsize=(20,10))
columns = 20
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.axis('off')
    #fig.tight_layout() 
    plt.imshow(image)